In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
import json

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

params = json.loads(open('../params.json', 'r').read())

# Date range
start = "2018-01-01"
end = "2023-11-01"

# Tickers of assets
industry_asset = json.loads(open('../stocks.json', 'r').read())
assets = []
for key, values in industry_asset.items():
    for value in values:
        assets.append(value)

# Downloading data
data = yf.download(assets, start=start, end=end, interval="1wk")

[*********************100%%**********************]  23 of 23 completed


In [2]:
from ta import trend, volume as volume_ta, momentum

THRESHOLD = 10

def get_trend(data_prices, i):
    trend = float(0.5)
    if 100 * (data_prices[i] / data_prices[i-1]) - 100 >= THRESHOLD:
        trend = float(1.0)
    elif 100 * (data_prices[i] / data_prices[i-1]) - 100 <= 0:
        trend = float(0.0)
    return trend

def get_trend_open(data_open, data_close, i):
    trend = float(0.5)
    if 100 * (data_open[i] / data_close[i-1]) - 100 >= THRESHOLD:
        trend = float(1.0)
    elif 100 * (data_open[i] / data_close[i-1]) - 100 <= 0:
        trend = float(0.0)
    return trend

def is_the_last_date(i, size):
    return i == size-1

indicators = {
    asset: {
        "date": [],
        "open": [],
        "close": [],
        "high": [],
        "low": [],
        "volume": [],
        "trend_open": [],
        "trend_close": [],
        "trend_high": [],
        "trend_low": [],
        "next_trend_open": [],
        "next_trend_close": [],
        "next_trend_high": [],
        "next_trend_low": [],
        "aroon": [],
        "aroon_down": [],
        "aroon_up": [],
        "cmf": [],
        "mfi": [],
        "ppo": [],
        "pvo": [],
        "rsi": [],
        "stc": [],
        "volume_rsi": [], 
        "williamsr": []
    }
    for asset in assets
}

dates = [d.to_pydatetime().strftime('%Y-%m-%d %H:%M:%S') for d in data.index]

for asset in assets:
    if int(dates[0][0:4]) > 2018:
        print(f"{int(dates[0][0:4])} is too new, skipping it.")
        continue

    high = data["High"][asset].astype(float)
    low = data["Low"][asset].astype(float)
    open = data["Open"][asset].astype(float)
    close = data["Close"][asset].astype(float)
    volume = data["Volume"][asset].astype(float)

    # AROON indicator
    aroon = trend.AroonIndicator(high, low).aroon_indicator()
    aroon_down = trend.AroonIndicator(high, low).aroon_down()
    aroon_up = trend.AroonIndicator(high, low).aroon_up()

    # CMF indicator
    indicator_calc = volume_ta.ChaikinMoneyFlowIndicator(high, low, close, volume)
    cmf = indicator_calc.chaikin_money_flow()

    # MFI indicator
    mfi = volume_ta.MFIIndicator(high, low, close, volume).money_flow_index()

    # PPO indicator
    indicator_calc = momentum.PercentagePriceOscillator(close)
    ppo = indicator_calc.ppo()

    # PVO indicator
    indicator_calc = momentum.PercentageVolumeOscillator(volume)
    pvo = indicator_calc.pvo()

    # RSI indicator
    rsi = momentum.RSIIndicator(close).rsi()

    # STC indicator
    stc = trend.STCIndicator(close, 12, 5, 3, 3, 3).stc()

    # VOLUME RSI indicator
    volume_rsi = momentum.RSIIndicator(volume).rsi()

    # WILLIAMS R indicator
    williamsr = momentum.WilliamsRIndicator(high, low, close).williams_r()

    data_len = len(williamsr)

    first_not_nan = True
    for i in range(1, data_len):
        if (
            not np.isnan(aroon[i]) and
            not np.isnan(aroon_up[i]) and
            not np.isnan(aroon_down[i]) and
            not np.isnan(cmf[i]) and
            not np.isnan(mfi[i]) and
            not np.isnan(ppo[i]) and
            not np.isnan(pvo[i]) and
            not np.isnan(rsi[i]) and
            not np.isnan(stc[i]) and
            not np.isnan(volume_rsi[i]) and
            not np.isnan(williamsr[i])
        ):
            if first_not_nan:
                first_not_nan = False
                continue

            trend_high = get_trend(high, i)
            trend_low = get_trend(low, i)
            trend_open = get_trend_open(open, close, i)
            trend_close = get_trend(close, i)

            next_trend_high = float(0.0) if is_the_last_date(i, data_len) else get_trend(high, i+1)
            next_trend_low = float(0.0) if is_the_last_date(i, data_len) else get_trend(low, i+1)
            next_trend_open = float(0.0) if is_the_last_date(i, data_len) else get_trend_open(open, close, i+1)
            next_trend_close = float(0.0) if is_the_last_date(i, data_len) else get_trend(close, i+1)

            indicators[asset]["date"].append(dates[i])
            indicators[asset]["high"].append(high[i])
            indicators[asset]["low"].append(low[i])
            indicators[asset]["open"].append(open[i])
            indicators[asset]["close"].append(close[i])
            indicators[asset]["volume"].append(close[i])
            indicators[asset]["aroon"].append(aroon[i])
            indicators[asset]["aroon_down"].append(aroon_down[i])
            indicators[asset]["aroon_up"].append(aroon_up[i])
            indicators[asset]["cmf"].append(cmf[i])
            indicators[asset]["mfi"].append(mfi[i])
            indicators[asset]["ppo"].append(ppo[i])
            indicators[asset]["pvo"].append(pvo[i])
            indicators[asset]["rsi"].append(rsi[i])
            indicators[asset]["stc"].append(stc[i])
            indicators[asset]["volume_rsi"].append(volume_rsi[i])
            indicators[asset]["williamsr"].append(williamsr[i])
            indicators[asset]["trend_high"].append(trend_high)
            indicators[asset]["trend_low"].append(trend_low)
            indicators[asset]["trend_open"].append(trend_open)
            indicators[asset]["trend_close"].append(trend_close)
            indicators[asset]["next_trend_high"].append(next_trend_high)
            indicators[asset]["next_trend_low"].append(next_trend_low)
            indicators[asset]["next_trend_open"].append(next_trend_open)
            indicators[asset]["next_trend_close"].append(next_trend_close)

In [3]:
from sklearn.preprocessing import MinMaxScaler

indicators_reshaped = {
    asset: {
        "open": [],
        "close": [],
        "high": [],
        "low": [],
        "volume": [],
        "trend_open": [],
        "trend_close": [],
        "trend_high": [],
        "trend_low": [],
        "next_trend_open": [],
        "next_trend_close": [],
        "next_trend_high": [],
        "next_trend_low": [],
        "aroon": [],
        "aroon_down": [],
        "aroon_up": [],
        "cmf": [],
        "mfi": [],
        "ppo": [],
        "pvo": [],
        "rsi": [],
        "stc": [],
        "volume_rsi": [], 
        "williamsr": []
    }
    for asset in assets
}

indicators_scaled = {
    asset: {
        "trend_open": [],
        "trend_close": [],
        "trend_high": [],
        "trend_low": [],
        "aroon": [],
        "aroon_down": [],
        "aroon_up": [],
        "cmf": [],
        "mfi": [],
        "ppo": [],
        "pvo": [],
        "rsi": [],
        "stc": [],
        "volume_rsi": [], 
        "williamsr": []
    }
    for asset in assets
}

indicators_scalers = {
    asset: {
        "trend_open": None,
        "trend_close": None,
        "trend_high": None,
        "trend_low": None,
        "aroon": None,
        "aroon_down": None,
        "aroon_up": None,
        "cmf": None,
        "mfi": None,
        "ppo": None,
        "pvo": None,
        "rsi": None,
        "stc": None,
        "volume_rsi": None, 
        "williamsr": None
    }
    for asset in assets
}

for asset in assets:
    # Reshape data
    for index, row in indicators[asset].items():
        if index not in ["date"]:
            indicators_reshaped[asset][index] = np.array(row)
            indicators_reshaped[asset][index] = indicators_reshaped[asset][index].reshape(-1, 1)

    # Create scalers
    for index, row in indicators_scalers[asset].items():
        indicators_scalers[asset][index] = MinMaxScaler(feature_range=(0, 1))

    # Discretize data
    for index, row in indicators_reshaped[asset].items():
        if index not in ["open", "close", "high", "low", "volume", "next_trend_open", "next_trend_close", "next_trend_high", "next_trend_low"]:
            indicators_scaled[asset][index] = indicators_scalers[asset][index].fit_transform(indicators_reshaped[asset][index])

In [7]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf

keras = tf.keras
layers = tf.keras.layers
activations = tf.keras.activations

# Receiving parameters
OUTPUT_MODEL_PATH = "trained-models/"
EPOCHS = 10000

for asset in assets:
    print(f"Processing high {asset}")

    processed_data = []
    ticker_date = indicators[asset]["date"][-1]
    number_of_lines = len(indicators_scaled[asset]["aroon"])
    for i in range(0, number_of_lines):
        if ticker_date == indicators[asset]["date"][i]:
            break

        processed_data.append([
            indicators[asset]["date"][i],
            [indicators[asset]["next_trend_high"][i]],
            indicators_scaled[asset]["trend_high"][i],
            indicators_scaled[asset]["trend_low"][i],
            indicators_scaled[asset]["trend_open"][i],
            indicators_scaled[asset]["trend_close"][i],
            indicators_scaled[asset]["aroon"][i],
            indicators_scaled[asset]["aroon_down"][i],
            indicators_scaled[asset]["aroon_up"][i],
            indicators_scaled[asset]["cmf"][i],
            indicators_scaled[asset]["mfi"][i],
            indicators_scaled[asset]["ppo"][i],
            indicators_scaled[asset]["pvo"][i],
            indicators_scaled[asset]["rsi"][i],
            indicators_scaled[asset]["stc"][i],
            indicators_scaled[asset]["williamsr"][i]
        ])

    # Model training
    train_input_data_x = np.array([d[2:] for d in processed_data])
    train_input_data_y = np.array([d[1:2] for d in processed_data])

    # Define Sequential model
    model = keras.Sequential()
    model.add(layers.Dense(len(train_input_data_x[0]), input_shape=(len(train_input_data_x[0]),), activation="sigmoid", name="layer1"))
    model.add(layers.Dense(20, activation="sigmoid", name="layer2"))
    model.add(layers.Dense(10, activation="sigmoid", name="layer3"))
    model.add(layers.Dense(1, name="layer4"))

    model.compile(
        optimizer=keras.optimizers.RMSprop(),  # Optimizer
        # Loss function to minimize
        loss=keras.losses.MeanSquaredError(),
    )

    model_trained = model.fit(
        train_input_data_x,
        train_input_data_y,
        batch_size=512,
        epochs=int(EPOCHS),
        verbose=0
    )

    model.save(f"{OUTPUT_MODEL_PATH}{asset}_high")

Processing high TIMS3.SA
INFO:tensorflow:Assets written to: trained-models/TIMS3.SA_high/assets


INFO:tensorflow:Assets written to: trained-models/TIMS3.SA_high/assets


Processing high VIVT3.SA
INFO:tensorflow:Assets written to: trained-models/VIVT3.SA_high/assets


INFO:tensorflow:Assets written to: trained-models/VIVT3.SA_high/assets


Processing high SUZB3.SA
INFO:tensorflow:Assets written to: trained-models/SUZB3.SA_high/assets


...

INFO:tensorflow:Assets written to: trained-models/GGBR3.SA_high/assets


Processing high CSNA3.SA
INFO:tensorflow:Assets written to: trained-models/CSNA3.SA_high/assets


INFO:tensorflow:Assets written to: trained-models/CSNA3.SA_high/assets


Processing high FESA4.SA
INFO:tensorflow:Assets written to: trained-models/FESA4.SA_high/assets


INFO:tensorflow:Assets written to: trained-models/FESA4.SA_high/assets


In [6]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf

keras = tf.keras
layers = tf.keras.layers
activations = tf.keras.activations

# Receiving parameters
OUTPUT_MODEL_PATH = "trained-models/"
EPOCHS = 10000

for asset in assets:
    print(f"Processing low {asset}")

    processed_data = []
    ticker_date = indicators[asset]["date"][-1]
    number_of_lines = len(indicators_scaled[asset]["aroon"])
    for i in range(0, number_of_lines):
        if ticker_date == indicators[asset]["date"][i]:
            break

        processed_data.append([
            indicators[asset]["date"][i],
            [indicators[asset]["next_trend_low"][i]],
            indicators_scaled[asset]["trend_high"][i],
            indicators_scaled[asset]["trend_low"][i],
            indicators_scaled[asset]["trend_open"][i],
            indicators_scaled[asset]["trend_close"][i],
            indicators_scaled[asset]["aroon"][i],
            indicators_scaled[asset]["aroon_down"][i],
            indicators_scaled[asset]["aroon_up"][i],
            indicators_scaled[asset]["cmf"][i],
            indicators_scaled[asset]["mfi"][i],
            indicators_scaled[asset]["ppo"][i],
            indicators_scaled[asset]["pvo"][i],
            indicators_scaled[asset]["rsi"][i],
            indicators_scaled[asset]["stc"][i],
            indicators_scaled[asset]["williamsr"][i]
        ])

    # Model training
    train_input_data_x = np.array([d[2:] for d in processed_data])
    train_input_data_y = np.array([d[1:2] for d in processed_data])

    # Define Sequential model
    model = keras.Sequential()
    model.add(layers.Dense(len(train_input_data_x[0]), input_shape=(len(train_input_data_x[0]),), activation="sigmoid", name="layer1"))
    model.add(layers.Dense(20, activation="sigmoid", name="layer2"))
    model.add(layers.Dense(10, activation="sigmoid", name="layer3"))
    model.add(layers.Dense(1, name="layer4"))

    model.compile(
        optimizer=keras.optimizers.RMSprop(),  # Optimizer
        # Loss function to minimize
        loss=keras.losses.MeanSquaredError(),
    )

    model_trained = model.fit(
        train_input_data_x,
        train_input_data_y,
        batch_size=512,
        epochs=int(EPOCHS),
        verbose=0
    )

    model.save(f"{OUTPUT_MODEL_PATH}{asset}_low")

Processing low TIMS3.SA
INFO:tensorflow:Assets written to: trained-models/TIMS3.SA_low/assets


INFO:tensorflow:Assets written to: trained-models/TIMS3.SA_low/assets


Processing low VIVT3.SA
INFO:tensorflow:Assets written to: trained-models/VIVT3.SA_low/assets


INFO:tensorflow:Assets written to: trained-models/VIVT3.SA_low/assets


Processing low SUZB3.SA
INFO:tensorflow:Assets written to: trained-models/SUZB3.SA_low/assets


...

INFO:tensorflow:Assets written to: trained-models/GGBR3.SA_low/assets


Processing low CSNA3.SA
INFO:tensorflow:Assets written to: trained-models/CSNA3.SA_low/assets


INFO:tensorflow:Assets written to: trained-models/CSNA3.SA_low/assets


Processing low FESA4.SA
INFO:tensorflow:Assets written to: trained-models/FESA4.SA_low/assets


INFO:tensorflow:Assets written to: trained-models/FESA4.SA_low/assets


In [5]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf

keras = tf.keras
layers = tf.keras.layers
activations = tf.keras.activations

# Receiving parameters
OUTPUT_MODEL_PATH = "trained-models/"
EPOCHS = 10000

for asset in assets:
    print(f"Processing open {asset}")

    processed_data = []
    ticker_date = indicators[asset]["date"][-1]
    number_of_lines = len(indicators_scaled[asset]["aroon"])
    for i in range(0, number_of_lines):
        if ticker_date == indicators[asset]["date"][i]:
            break

        processed_data.append([
            indicators[asset]["date"][i],
            [indicators[asset]["next_trend_open"][i]],
            indicators_scaled[asset]["trend_high"][i],
            indicators_scaled[asset]["trend_low"][i],
            indicators_scaled[asset]["trend_open"][i],
            indicators_scaled[asset]["trend_close"][i],
            indicators_scaled[asset]["aroon"][i],
            indicators_scaled[asset]["aroon_down"][i],
            indicators_scaled[asset]["aroon_up"][i],
            indicators_scaled[asset]["cmf"][i],
            indicators_scaled[asset]["mfi"][i],
            indicators_scaled[asset]["ppo"][i],
            indicators_scaled[asset]["pvo"][i],
            indicators_scaled[asset]["rsi"][i],
            indicators_scaled[asset]["stc"][i],
            indicators_scaled[asset]["williamsr"][i]
        ])

    # Model training
    train_input_data_x = np.array([d[2:] for d in processed_data])
    train_input_data_y = np.array([d[1:2] for d in processed_data])

    # Define Sequential model
    model = keras.Sequential()
    model.add(layers.Dense(len(train_input_data_x[0]), input_shape=(len(train_input_data_x[0]),), activation="sigmoid", name="layer1"))
    model.add(layers.Dense(20, activation="sigmoid", name="layer2"))
    model.add(layers.Dense(10, activation="sigmoid", name="layer3"))
    model.add(layers.Dense(1, name="layer4"))

    model.compile(
        optimizer=keras.optimizers.RMSprop(),  # Optimizer
        # Loss function to minimize
        loss=keras.losses.MeanSquaredError(),
    )

    model_trained = model.fit(
        train_input_data_x,
        train_input_data_y,
        batch_size=512,
        epochs=int(EPOCHS),
        verbose=0
    )

    model.save(f"{OUTPUT_MODEL_PATH}{asset}_open")

Processing open TIMS3.SA
INFO:tensorflow:Assets written to: trained-models/TIMS3.SA_open/assets


INFO:tensorflow:Assets written to: trained-models/TIMS3.SA_open/assets


Processing open VIVT3.SA
INFO:tensorflow:Assets written to: trained-models/VIVT3.SA_open/assets


INFO:tensorflow:Assets written to: trained-models/VIVT3.SA_open/assets


Processing open SUZB3.SA
INFO:tensorflow:Assets written to: trained-models/SUZB3.SA_open/assets


...

INFO:tensorflow:Assets written to: trained-models/GGBR3.SA_open/assets


Processing open CSNA3.SA
INFO:tensorflow:Assets written to: trained-models/CSNA3.SA_open/assets


INFO:tensorflow:Assets written to: trained-models/CSNA3.SA_open/assets


Processing open FESA4.SA
INFO:tensorflow:Assets written to: trained-models/FESA4.SA_open/assets


INFO:tensorflow:Assets written to: trained-models/FESA4.SA_open/assets


In [ ]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf

keras = tf.keras
layers = tf.keras.layers
activations = tf.keras.activations

# Receiving parameters
OUTPUT_MODEL_PATH = "trained-models/"
EPOCHS = 10000

for asset in assets:
    print(f"Processing close {asset}")

    processed_data = []
    ticker_date = indicators[asset]["date"][-1]
    number_of_lines = len(indicators_scaled[asset]["aroon"])
    for i in range(0, number_of_lines):
        if ticker_date == indicators[asset]["date"][i]:
            break

        processed_data.append([
            indicators[asset]["date"][i],
            [indicators[asset]["next_trend_close"][i]],
            indicators_scaled[asset]["trend_high"][i],
            indicators_scaled[asset]["trend_low"][i],
            indicators_scaled[asset]["trend_open"][i],
            indicators_scaled[asset]["trend_close"][i],
            indicators_scaled[asset]["aroon"][i],
            indicators_scaled[asset]["aroon_down"][i],
            indicators_scaled[asset]["aroon_up"][i],
            indicators_scaled[asset]["cmf"][i],
            indicators_scaled[asset]["mfi"][i],
            indicators_scaled[asset]["ppo"][i],
            indicators_scaled[asset]["pvo"][i],
            indicators_scaled[asset]["rsi"][i],
            indicators_scaled[asset]["stc"][i],
            indicators_scaled[asset]["williamsr"][i]
        ])

    # Model training
    train_input_data_x = np.array([d[2:] for d in processed_data])
    train_input_data_y = np.array([d[1:2] for d in processed_data])

    # Define Sequential model
    model = keras.Sequential()
    model.add(layers.Dense(len(train_input_data_x[0]), input_shape=(len(train_input_data_x[0]),), activation="sigmoid", name="layer1"))
    model.add(layers.Dense(20, activation="sigmoid", name="layer2"))
    model.add(layers.Dense(10, activation="sigmoid", name="layer3"))
    model.add(layers.Dense(1, name="layer4"))

    model.compile(
        optimizer=keras.optimizers.RMSprop(),  # Optimizer
        # Loss function to minimize
        loss=keras.losses.MeanSquaredError(),
    )

    model_trained = model.fit(
        train_input_data_x,
        train_input_data_y,
        batch_size=512,
        epochs=int(EPOCHS),
        verbose=0
    )

    model.save(f"{OUTPUT_MODEL_PATH}{asset}_close")